# Read the data

Might need to re-factor the source, encoding = 'latin-1' is undesirable.

In [13]:
pluri_patients = pd.read_csv('../data/raw/pluri_patients_2020.csv', sep = ';')

In [14]:
pluri_patients.head()

,edad,sexo,n_urg,n_hosp,n_cx,n_ce_med_general,n_ce_med_especializada,estancia_sala,estancia_uce,estancia_uci,...,tasa_de_filtración_glomerular_tfg,estadio_de_la_enfermedad_renal,microalbuminuria,tsh,clase_funcional,creatinina_2_consulta,tasa_de_filtración_glomerular_tfg3,cambio_de_tfg,úlcera_de_pie_diabético,estado_vital
0,29,F,1,1,0,3,9,2,0,0,...,NaN,Estadio 0,NaN,NaN,No clasificado,NaN,NaN,NaN,No,vivo
1,67,F,1,0,0,0,1,0,0,0,...,"60,13",Estadio 2,"3,02","5,770",Clase funcional 2B,"0,86","60,13","0,00",No,vivo
2,83,M,1,0,0,4,3,0,0,0,...,NaN,Estadio 0,NaN,NaN,No clasificado,NaN,NaN,NaN,No,vivo
3,66,M,1,1,1,0,2,13,0,0,...,"58,09",Estadio 3A,"7,00","1,170",Clase funcional 4,"0,00","0,00","0,00",No,fallecido
4,62,F,2,0,0,2,8,0,0,0,...,"107,76",Estadio 1,"68,00","2,890",Clase funcional 2B,"0,00","0,00","0,00",No,vivo


In [19]:
profile = ProfileReport(pluri_patients, minimal=True)

In [18]:
# to write the report
profile.to_file("../reports/pluri_patients_minimal.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]